In [16]:
import logging
import os
import openai

from typing import List

from actionweaver import ActionHandlerMixin, action
from actionweaver.cache import preserve_original_signature
from actionweaver.llms.openai.chat import OpenAIChatCompletion
from actionweaver.llms.openai.tokens import TokenUsageTracker

from pydantic import BaseModel

openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
logging.basicConfig(
    filename='agent.log',
    filemode='a',
    format='%(asctime)s.%(msecs)04d %(levelname)s {%(module)s} [%(funcName)s] %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S'
)

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

In [4]:
class LLMAgent(ActionHandlerMixin):
    def __init__(self, logger):
        self.logger = logger
        self.token_tracker = TokenUsageTracker(budget=2000, logger=logger)
        self.llm = OpenAIChatCompletion("gpt-3.5-turbo-0613", token_usage_tracker = self.token_tracker, logger=logger)
        self.messages = [{"role": "system", "content": "You're a helpful assistant."}]



class AgentV0(LLMAgent):
    def __call__(self, text):
        self.messages += [{"role": "user", "content":text}]
        response = self.llm.create(messages=self.messages, scope='global')
        return response

    @action(name="GetCurrentTime", scope="global")
    def get_current_time(self) -> str:
        """
        Use this for getting the current time in the specified time zone.
        
        :return: A string representing the current time in the specified time zone.
        """
        import datetime
        current_time = datetime.datetime.now()
        
        return f"The current time is {current_time}"

    import time

    @action(name="Sleep", scope="global")
    def sleep(self, seconds: int) -> str:
        """
        Introduces a sleep delay of the specified seconds and returns a message.
    
        Args:
            seconds (int): The duration to sleep in seconds.
    
        Returns:
            str: A message indicating the completion of the sleep.
        """
        import time
        time.sleep(seconds)
        return f"Now I wake up after sleep {seconds} seconds."


agent = AgentV0(logger)

In [29]:
agent("have you slept")

'Yes, I have slept for 1 second.'

In [15]:
agent("how long have you slept")

'I slept for 60 seconds, which is equivalent to 1 minute.'

In [11]:
class FileUtility(LLMAgent):
    @action(name="FileHandler", scope="global")
    def handle_file(self, instruction: str) -> str:
        """
        Handles user queries related to file operations.
    
        Args:
            instruction (str): The user's instruction about file handling.
    
        Returns:
            str: The response to the user's question.
        """
        return self.llm.create(messages=[{'role': 'user', 'content': instruction}], scope='file')
        

    @action(name="ListFiles", scope="file")
    def list_all_files_in_repo(self, repo_path: str ='.') -> List:
        """
        Lists all the files in the given repository.
    
        :param repo_path: Path to the repository. Defaults to the current directory.
        :return: List of file paths.
        """

        logger.info(f"list_all_files_in_repo: {repo_path}")
        
        file_list = []
        for root, _, files in os.walk(repo_path):
            for file in files:
                file_list.append(os.path.join(root, file))
            break
        return file_list

    @action(name="ReadFile", scope="file")
    def read_from_file(self, file_path: str) -> str:
        """
        Reads the content of a file and returns it as a string.
    
        :param file_path: The path to the file that needs to be read.
        :return: A string containing the content of the file.
        """
        logger.info(f"read_from_file: {file_path}")
        
        with open(file_path, 'r') as file:
            content = file.read()
        return f"The file content: \n{content}"

    @action(name="WriteFile", scope="file")
    def write_to_file(self, file_path: str, content: str) -> str:
        """
        Writes the given content to a file.
        
        :param file_path: The path to the file where the content should be written.
        :param content: The content to be written to the file.
        """
        try:
            with open(file_path, 'a') as file:
                file.write(content)
            return "Content successfully appended to the file."
        except Exception as e:
            return f"An error occurred while appending to the file: {str(e)}"
        


class AgentV1(AgentV0, FileUtility):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)


In [13]:
agent = AgentV1(logger)

In [10]:
agent("thanks")

"You're welcome! If you have any more questions, feel free to ask."

In [14]:
class LangChainTools(LLMAgent):
    @action(name="GoogleSearch", scope="global")
    def google_search(self, query: str) -> str:
        """
        Perform a Google search using the provided query. 
        
        This action requires `langchain` and `google-api-python-client` installed, and GOOGLE_API_KEY, GOOGLE_CSE_ID environment variables.
        See https://python.langchain.com/docs/integrations/tools/google_search.

        :param query: The search query to be used for the Google search.
        :return: The search results as a string.
        """
        from langchain.utilities import GoogleSearchAPIWrapper

        search = GoogleSearchAPIWrapper()
        return search.run(query)
    
class AgentV2(AgentV1, LangChainTools, FileUtility):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)


In [68]:
agent = AgentV2(logger)

In [71]:
agent("""what is the coordinate of capitol of Nigeria""")

'The capital of Nigeria is Abuja. The coordinates for Abuja are approximately 9.0579° N latitude and 7.4951° E longitude.'

In [35]:
class Place(BaseModel):
    lat: float
    lng: float
    description: str

class Folium(ActionHandlerMixin):
    @action(name="ShowMap", scope="global")
    def show_map(self, places: List[Place]) -> str:
        """
        Display a map with the provided latitude and longitude coordinates.
        
        This action requires proper integration with map services and may require API keys or authentication.
        Ensure that relevant documentation and dependencies are properly set up.

        This method requires `folium` installed
        """
        import folium

        # Calculate the center of all places
        avg_lat = sum(place['lat'] for place in places) / len(places)
        avg_lng = sum(place['lng'] for place in places) / len(places)
    
        # Create a folium Map centered at the average latitude and longitude
        m = folium.Map(location=[avg_lat, avg_lng])
    
        # Add markers for each place
        for place in places:
            folium.Marker([place['lat'], place['lng']], tooltip=place['description']).add_to(m)
    
        # Display the map
        display(m)
        return "Map has been displayed"


class AgentV2(AgentV0, Folium, LangChainTools):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

agent = AgentV2(logger)

In [39]:
agent("show me its Headquarter on the map")

TokenUsageTrackerException: Token budget exceeded. Budget: 2000, Usage: {'prompt_tokens': 2103, 'completion_tokens': 125, 'total_tokens': 2228}